In [ ]:
# Import required libraries
import requests
import json
import pandas as pd
from tradingview_ta import TA_Handler, Interval, Exchange

# Step 1: Set up the TradingView screener endpoint
url = "https://scanner.tradingview.com/india/scan"
headers = {
    "User-Agent": "Mozilla/5.0",  # Mimic a browser user-agent
    "Content-Type": "application/json"  # Data being sent is JSON
}

# Step 2: Create a payload to request stock data (OHLC + Volume)
payload = {
  "filter": [
    {"left": "name", "operation": "nempty"}  # Get all stocks where name is not empty
  ],
  "symbols": {
    "query": {"types": []},  # No specific filter on types
    "tickers": []            # No specific tickers — fetch all
  },
  "columns": ["name", "close", "open", "high", "low", "volume"]  # Fields to retrieve
}

# Step 3: Send POST request to get the data
response = requests.post(url, headers=headers, data=json.dumps(payload))

# Step 4: Parse the JSON response
data = response.json()

# Step 5: Extract actual stock data and convert it into a DataFrame
results = [x['d'] for x in data['data']]
df = pd.DataFrame(results, columns=["name", "close", "open", "high", "low", "volume"])

# Step 6: Ask user to input stock symbol and filter matching rows
stock_name = input("Enter a symbol name: ").upper()
print(df[df['name'].str.contains(stock_name)])




# Step 7: Create a handler for a specific stock using tradingview_ta
handler = TA_Handler(
    symbol= stock_name,               # Stock symbol
    exchange="NSE",                  # Exchange name
    screener="india",                # Region screener (India)
    interval=Interval.INTERVAL_1_DAY  # Timeframe for technical analysis
)

# Step 8: Get technical analysis using the handler
try:
    analysis = handler.get_analysis()

    # Step 9: Extract all indicator values
    indicators = analysis.indicators

    # Step 10: Print selected technical indicators
    print("Technical Indicators for RELIANCE:")
    print(f"RSI: {indicators.get('RSI')}")
    print(f"MACD: {indicators.get('MACD.macd')}")
    print(f"MACD Signal: {indicators.get('MACD.signal')}")
    print(f"SMA 20: {indicators.get('SMA20')}")
    print(f"SMA 50: {indicators.get('SMA50')}")
    print(f"SMA 200: {indicators.get('SMA200')}")
    
    # Step 11: Print the summary recommendation (Buy/Sell/Neutral)
    print(f"Summary Recommendation: {analysis.summary['RECOMMENDATION']}")

except Exception as e:
    # Step 12: Handle and display errors (e.g., if the symbol is invalid or no data)
    print(f"Error getting data: {e}")


In [2]:
import requests
import json
import pandas as pd
from tradingview_ta import TA_Handler, Interval

# STEP 1: Function to search OHLC from TradingView screener
def search_stock_ohlc():
    url = "https://scanner.tradingview.com/india/scan"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Content-Type": "application/json"
    }

    payload = {
        "filter": [
            {"left": "name", "operation": "nempty"} 
        ],
        "symbols": {
            "query": {"types": []},
            "tickers": []
        },
        "columns": ["name", "close", "open", "high", "low", "volume"]
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))
    data = response.json()

    results = [x['d'] for x in data['data']]
    df = pd.DataFrame(results, columns=["name", "close", "open", "high", "low", "volume"])

    return df

# STEP 2: Function to get technical indicators using tradingview_ta
def get_indicators(symbol):
    try:
        handler = TA_Handler(
            symbol=symbol,
            exchange="NSE",
            screener="india",
            interval=Interval.INTERVAL_1_DAY
        )
        analysis = handler.get_analysis()
        indicators = analysis.indicators

        print(f"\n Technical Indicators for {symbol}:")
        print(f"RSI: {indicators.get('RSI')}")
        print(f"MACD: {indicators.get('MACD.macd')}")
        print(f"MACD Signal: {indicators.get('MACD.signal')}")
        print(f"SMA 20: {indicators.get('SMA20')}")
        print(f"SMA 50: {indicators.get('SMA50')}")
        print(f"SMA 200: {indicators.get('SMA200')}")
        print(f"Summary Recommendation: {analysis.summary['RECOMMENDATION']}\n")
    except Exception as e:
        print(f" Error getting data for {symbol}: {e}\n")

# Main loop
ohlc_df = search_stock_ohlc()
while True:
    user_input = input("Enter a symbol name (or type 'exit' to quit): ").upper()
    if user_input == "EXIT":
        print(" Exiting the program.")
        break
    
    # Filter OHLC
    match = ohlc_df[ohlc_df['name'].str.contains(user_input)]
    if match.empty:
        print(" Symbol not found in OHLC data.\n")
    else:
        print("\n OHLC Data:")
        print(match)

    # Show indicators
    get_indicators(user_input)


Enter a symbol name (or type 'exit' to quit): samhi hotels
 Symbol not found in OHLC data.

 Error getting data for SAMHI HOTELS: Exchange or symbol not found.

Enter a symbol name (or type 'exit' to quit): lemon tree hotels
 Symbol not found in OHLC data.

 Error getting data for LEMON TREE HOTELS: Exchange or symbol not found.

Enter a symbol name (or type 'exit' to quit): recl

 OHLC Data:
        name   close   open    high     low    volume
1013  RECLTD  400.75  408.8  408.80  397.75    575381
2374   RRECL  281.30  281.3  281.30  281.30      7861
6178  RECLTD  400.75  408.0  408.25  397.65  15105541
 Error getting data for RECL: Exchange or symbol not found.

Enter a symbol name (or type 'exit' to quit): irfc

 OHLC Data:
      name   close   open    high     low    volume
3992  IRFC  132.05  135.3  135.55  131.05   1259937
6530  IRFC  132.03  135.5  135.50  130.98  11164552

 Technical Indicators for IRFC:
RSI: 40.16397908511735
MACD: -1.3404446470713935
MACD Signal: -0.986432385